In [ ]:
import backtrader as bt
import datetime

import pandas as pd

In [ ]:
# mongo feed 使用数据库作为数据源
from mongofeed import MongoData

feed = MongoData(
    db='stock_etf',
    dataname='etf_510300',
    fromdate=datetime.datetime(2016,1,1),
    todate=datetime.datetime(2019,12,31), 
)

In [ ]:
class KDJ(bt.Indicator):
    lines = ('K','D','J')

    params = (
        ('period', 9),
        ('period_dfast', 3),
        ('period_dslow', 3),
    )
    
    plotlines = dict(
        J=dict(
            _fill_gt=('K', ('red', 0.50)),
            _fill_lt=('K', ('green', 0.50)),
        )
    )
    
    def __init__(self):
        # Add a KDJ indicator
        self.kd = bt.indicators.StochasticFull(
            self.data,
            period = self.p.period,
            period_dfast = self.p.period_dfast,
            period_dslow = self.p.period_dslow,
        )
        
        self.l.K = self.kd.percD
        self.l.D = self.kd.percDSlow
        self.l.J = self.K*3 - self.D*2
        
        

In [ ]:
class KDJStrategy(bt.Strategy):
    params = (
        ('period', 9),
        ('period_dfast', 3),
        ('period_dslow', 3),
    )
    
        
    def __init__(self):
        
        # use self defind a KDJ indicator
        self.kd = KDJ(
            self.data0,
            period = self.p.period,
            period_dfast = self.p.period_dfast,
            period_dslow = self.p.period_dslow,
        )
    
        self.crossover = bt.indicators.CrossOver(self.kd.K, self.kd.D, plot=False)
        #self.above = bt.And(self.macd.macd>0.0, self.macd.macdsignal>0.0)
        
        self.buy_signal = (self.crossover==1)
        self.sell_signal = (self.crossover==-1)
        # To keep track of pending orders
        self.order = None
        
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        if order.status in [order.Completed, order.Canceled, order.Margin, order.Rejected]:
            # Write down: no pending order
            self.order = None

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if ...
            if self.buy_signal[0]:
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
        else:
            # Already in the market ... we might sell
            if self.sell_signal[0]:
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
cerebro = bt.Cerebro()


cerebro.adddata(feed, name= 'etf300')
cerebro.addstrategy(KDJStrategy)


# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

# 以发出信号当日收盘价成交
cerebro.broker.set_coc(True)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.AllInSizerInt, percents=99)

print('Starting Portfolio Value: {:.2f}'.format(cerebro.broker.getvalue()))

cerebro.addanalyzer(bt.analyzers.SQN)

result = cerebro.run()

print('Ending Portfolio Value: {:.2f}'.format(cerebro.broker.getvalue()))

In [ ]:
ana = result[0].analyzers.sqn.get_analysis()
print("sqn: {:.3f}, trades:{:d}".format(ana['sqn'],ana['trades']))

In [ ]:
cerebro.plot(iplot=False)